In [1]:
import pandas as pd
import requests
import time
import json
df=pd.read_csv('../data/activity_counts.csv')

You may change your own github_token in the config.py. You can just refer here https://github.com/settings/tokens. click `personal access tokens` --> `generate new token`

In [2]:
from config import github_token

In [3]:
github_token

'adc14b4d4916460707774abfff3a8ba989192131'

In [4]:
df

,Course_Code,Student_Name,github_link,remark,data-count,github_id
0,JOUR2106,Hu Pili,https://github.com/hupili,Lecturer,94,hupili
1,JOUR2106,DONG Xinyue,https://github.com/Felicity2724,Enrolled,75,Felicity2724
2,JOUR2106,Wang Ziwei,https://github.com/ivywang958,Helper,53,ivywang958
3,JOUR2106,Holly Chik,https://github.com/hollychik,Enrolled,49,hollychik
4,JOUR2106,LI Yun,https://github.com/Ly-Iris,Enrolled,45,Ly-Iris
5,JOUR2106,ZHAO Minzhu,https://github.com/MindyZHAOMinzhu,Enrolled,43,MindyZHAOMinzhu
6,JOUR2106,Gursimran Hans,https://github.com/ghans19,Enrolled,32,ghans19
7,JOUR2106,Li Yuquan,https://github.com/WindyLI75,Enrolled,31,WindyLI75
8,JOUR2106,NG Yee Sin,https://github.com/tiffanynys,Enrolled,29,tiffanynys
9,JOUR2106,Zhao Zoya,https://github.com/zoyazhao1996,Audit,25,zoyazhao1996


In [5]:
users = df['github_id'].tolist()

In [6]:
# only can get 300 maximum 
def get_details(url):
    r = requests.get(url,headers={
    'User-Agent':'Mozilla/5.0',
    'Authorization':'token {0}'.format(github_token),
    'Content-Type':'application/json',
    'method':'GET',
    'Accept':'application/json'
    })
    content = r.json()
    json_str = json.dumps(content)
    
    with open('../user-activities-json/{0}-page{1}&per_page=100.json'.format(user,i), "w") as f:
        f.write(json_str)
        f.close()

    commits = []
    for item in content:
        commit = {}
        commit['time'] = item['created_at']
        commit['repo'] = item['repo']['name']
        commit['users'] = item['actor']['login']
        commit['type'] = item['type']
        if item['type'] == 'CreateEvent':
            commit['description'] = item['repo']['url']
        elif item['type'] == 'IssuesEvent':
            commit['description'] = item['payload']['action']
        elif item['type'] == 'PushEvent':
            commit['description'] = item['repo']['url']
        elif item['type'] == 'WatchEvent':
            commit['description'] = item['repo']['url']
        else:
            commit['description'] = None
        
        commits.append(commit)
    return commits

In [7]:
all_commits = []
for user in users:
    one_user_commits=[]
    for i in range(1,4):
        url = 'https://api.github.com/users/{0}/events/public?page={1}&per_page=100'.format(user,i)
        commits = get_details(url)
        one_user_commits.extend(commits)
    all_commits.extend(one_user_commits)

In [8]:
df2 = pd.DataFrame(all_commits)

In [9]:
import csv
with open('../data/user-activity-details.csv','a') as f:
    df2.to_csv(f, header=None,index=None)

In [10]:
df3 = pd.read_csv('../data/user-activity-details.csv')

In [11]:
len(df3)

4761

In [12]:
df3 = df2.drop_duplicates()

In [13]:
len(df3)

2415

In [14]:
df3.to_csv('../data/user-activity-details.csv',index=None)

In [15]:
df4 = df3.pivot_table(index=['users','type'],values = 'time',aggfunc='count')
df4

time
users           type                    
CheungYanYi     CreateEvent            2
                ForkEvent              1
                PushEvent            110
FangFangYC      CreateEvent            2
                ForkEvent              2
                PushEvent             76
Felicity2724    CreateEvent            4
                ForkEvent              1
                MemberEvent            3
                PushEvent            161
                WatchEvent             2
JimmyDingyiming CreateEvent            2
                ForkEvent              2
                IssuesEvent            2
                PushEvent             68
Lewis2019       CreateEvent            2
                ForkEvent              2
                PushEvent              1
Ly-Iris         CreateEvent            2
                ForkEvent              4
                IssueCommentEvent      2
                IssuesEvent            2
                PushEvent            104
Midorillusion   CreateEvent            2
                ForkEvent              1
                PushEvent             59
MindyZHAOMinzhu CreateEvent            8
                ForkEvent              2
                IssueCommentEvent      1
                IssuesEvent            1
...                                  ...
hupili          PullRequestEvent      11
                PushEvent             43
                WatchEvent            21
ivyWANG958      CreateEvent            4
                ForkEvent              6
                IssueCommentEvent      9
                IssuesEvent            7
                PullRequestEvent      11
                PushEvent             52
                WatchEvent            15
majormoke       CreateEvent            2
                PushEvent              4
minhanhngg      CreateEvent            2
                ForkEvent              1
                PushEvent             90
rayliurui       CreateEvent            2
                ForkEvent              4
                PushEvent             75
sinyungli       ForkEvent              2
                PushEvent            190
tarzankuang     CommitCommentEvent     2
                CreateEvent            3
                ForkEvent              3
                PushEvent             27
tiffanynys      CreateEvent            1
                ForkEvent              1
                PushEvent             90
zoyazhao1996    CreateEvent            3
                ForkEvent              2
                PushEvent             29

[98 rows x 1 columns]